In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd

In [2]:
data2001 = pd.read_csv('./pems11/d11_text_station_5min_2018_02_28.txt.gz',header=None, usecols=range(12))
# Assign column names based on the provided headers
column_names = [
    "Timestamp", "Station", "District", "Freeway #", 
    "Direction of Travel", "Lane Type", "Station Length", 
    "Samples", "% Observed", "Total Flow", "Avg Occupancy", "Avg Speed"
]

# Assign column names to the dataframe
data2001.columns = column_names
data2001

,Timestamp,Station,District,Freeway #,Direction of Travel,Lane Type,Station Length,Samples,% Observed,Total Flow,Avg Occupancy,Avg Speed
0,02/28/2018 00:00:00,1100313,11,5,N,FR,NaN,10,100,6.0,NaN,NaN
1,02/28/2018 00:00:00,1100323,11,5,N,FR,NaN,20,0,NaN,NaN,NaN
2,02/28/2018 00:00:00,1100326,11,5,N,FR,NaN,20,100,10.0,NaN,NaN
3,02/28/2018 00:00:00,1100330,11,5,N,FR,NaN,20,100,2.0,NaN,NaN
4,02/28/2018 00:00:00,1100333,11,5,N,FR,NaN,10,100,6.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
433435,02/28/2018 23:55:00,1126770,11,52,W,ML,0.424,24,100,47.0,0.0132,67.1
433436,02/28/2018 23:55:00,1126790,11,805,S,ML,0.805,36,100,109.0,0.0268,64.1
433437,02/28/2018 23:55:00,1126791,11,805,S,FR,NaN,9,100,8.0,NaN,NaN
433438,02/28/2018 23:55:00,1126792,11,805,S,HV,2.935,9,100,0.0,0.0000,65.1


In [3]:
comm = np.load('pems11_comm.npy')
comm

array([1100313, 1100326, 1100330, 1100340, 1100348, 1100353, 1100363,
       1100369, 1100372, 1100377, 1100381, 1100384, 1100387, 1100390,
       1100393, 1100396, 1100407, 1100416, 1100434, 1100451, 1100454,
       1100461, 1100475, 1100478, 1100486, 1100491, 1100504, 1100524,
       1100530, 1100533, 1100534, 1100538, 1100542, 1100549, 1100553,
       1100560, 1100568, 1100571, 1100575, 1100578, 1100592, 1100595,
       1100598, 1100601, 1100604, 1100607, 1100697, 1100703, 1100706,
       1100711, 1100714, 1100729, 1100732, 1100737, 1100741, 1100745,
       1100750, 1100757, 1100776, 1100781, 1100787, 1100796, 1100802,
       1100805, 1100810, 1100813, 1100850, 1100860, 1100863, 1100865,
       1100868, 1107608, 1107611, 1108285, 1108286, 1108287, 1108288,
       1108289, 1108290, 1108291, 1108292, 1108293, 1108294, 1108295,
       1108296, 1108297, 1108299, 1108300, 1108301, 1108302, 1108303,
       1108304, 1108305, 1108306, 1108307, 1108308, 1108309, 1108310,
       1108311, 1108

In [4]:
import pandas as pd
import calendar
from datetime import date, timedelta

def read_and_process_data(file_path):
    try:
        # Read the CSV file, specifying the usecols parameter to only load the columns of interest
        data = pd.read_csv(
            file_path, 
            header=None, 
            usecols=[0, 1, 9],  # Column indexes for Timestamp, Station, and Total Flow
            names=['Timestamp', 'Station', 'Avg Flow'],  # Assigning column names
            compression='gzip'
        )

        # Convert the Timestamp column to datetime format and set it as the index
        data['Timestamp'] = pd.to_datetime(data['Timestamp'])
        data.set_index('Timestamp', inplace=True)

        # Pivot the table to get Stations as columns and Total Flow as cell values
        pivot_data = data.pivot(columns='Station', values='Avg Flow')

        return pivot_data

    except EOFError:
        print(f"Error processing file: {file_path}")
        return None

# Read the data for February 28 to get the column names (Station IDs)
# feb_28_data = read_and_process_data('./pems03/d03_text_station_5min_2018_02_28.txt.gz')
# feb_28_columns = feb_28_data.columns

# Initialize a list to hold the data for all days
all_data = []

start_date = date(2002, 9, 1)  # 开始日期
# end_date = date(2012, 10, 2)   # 结束日期
end_date = date(2024, 3, 20)   # 结束日期
current_date = start_date
all_data = []                  # 存储所有数据的列表

while current_date <= end_date:
    year = current_date.year
    month = current_date.month
    day = current_date.day
    file_path = f'./pems11/d11_text_station_5min_{year}_{month:02d}_{day:02d}.txt.gz'
#     if year==2012 and month==10 and day==1:
#         continue
    # 检查文件是否存在
    if not os.path.exists(file_path):
        print(f"File not found: {file_path}, skipping...")
        current_date += timedelta(days=1)
        continue
    
    day_data = read_and_process_data(file_path)
    
    # 使用字典存储新列，以便一次性添加到数据帧
    new_columns = {}
    for col in comm:
        if col not in day_data:
            new_columns[col] = 0  # 假设使用0填充缺失的列

    # 使用 pd.concat 添加所有新列
    if new_columns:
        new_data = pd.DataFrame(new_columns, index=day_data.index)
        day_data = pd.concat([day_data, new_data], axis=1)

    # 将列重新排序以匹配2月28日的顺序
    day_data = day_data[comm]
    
    all_data.append(day_data)
    
    current_date += timedelta(days=1)

# Concatenate all daily data into a single DataFrame
combined_data = pd.concat(all_data)

# Fill any remaining missing values with 0
combined_data.fillna(0, inplace=True)

# The combined_data DataFrame now holds the merged data for January 1 to February 28
# with columns aligned to February 28 and missing values filled with 0
# Note: The code execution is commented out to prevent execution in this environment.
# combined_data.head()
combined_data.to_csv('pems11_all_common_flow.csv')

In [ ]:
combined_data

In [1]:
import pandas as pd
pems11 = pd.read_csv('pems11_all_common_flow.csv')

# Convert the 'date' column to datetime
pems11['date'] = pd.to_datetime(pems11['date'])

# Set the 'date' column as the index
pems11.set_index('date', inplace=True)

# Resample to hourly data and aggregate using sum
pems11_hourly = pems11.resample('H').sum()

# Reset the index if you want the 'date' column back
pems11_hourly.reset_index(inplace=True)
pems11_hourly.to_csv('pems11_h.csv',index=False)


# Convert the 'date' column to datetime
pems11 = pd.read_csv('pems11_all_common_flow.csv')
pems11['date'] = pd.to_datetime(pems11['date'])

# Set the 'date' column as the index
pems11.set_index('date', inplace=True)

# Resample to hourly data and aggregate using sum
pems11_daily = pems11.resample('D').sum()

# Reset the index if you want the 'date' column back
pems11_daily.reset_index(inplace=True)
pems11_daily.to_csv('pems11_d.csv',index=False)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_47676\3132681090.py:11: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  pems11_hourly = pems11.resample('H').sum()
